




This notebook was prepared for the Applied Data Science Capstone, as part of the final assignment "The Battle of Neighborhoods".

We will try to find the best neighbourhood(s) to open an italian restaurant in Manhattan, using as basis the existing proportion of italian restaurants in each of Manhattan's different neibourhoods. Italian resturants are very populat in Manhattan and we'll assume that a neighbourhood that doesn't feature one in the TOP10 restaurants/food categories shows "untapped" potential for a new Italian restaurant.





# Setup

In [1]:
import pandas as pd
import requests
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.style.use(['seaborn'])

CLIENT_ID = 'STQGUAFY5FAFEPPIK4B4M4XB55RIM3CCP422T0A44QDE3ZVA' # your Foursquare ID
CLIENT_SECRET = 'QZNFTY3TTGC2ANQXJAG4XCWBTOQUY41AQUJYPBKCBNHTIVKA' # your Foursquare Secret
ACCESS_TOKEN = 'KLXKJPQWXKDID42D2PD1Z5KWUVXCSQUYYKBTASTBLJQB05DD' # your FourSquare Access Token
VERSION = '20210401'
LIMIT = 500
radius = 500

In [2]:


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium 
import folium # map rendering library

print('Libraries imported.')





/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.3 MB/s  eta 0:00:01
Libraries imported.


In [3]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    


    
    

Data downloaded!


# Creating a first map of all Manhattan's neighbourhoods for reference

In [19]:


manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


# Getting the information on restaurants/food places from Foursquare and organizing the data for each neighbourhood

In [5]:


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        categoryId = '4d4b7105d754a06374d81259'
        urlfood = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
            
        # make the GET request
        results = requests.get(urlfood).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [7]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,25,25,25,25,25,25
Carnegie Hill,68,68,68,68,68,68
Central Harlem,44,44,44,44,44,44
Chelsea,89,89,89,89,89,89
Chinatown,100,100,100,100,100,100
Civic Center,77,77,77,77,77,77
Clinton,100,100,100,100,100,100
East Harlem,51,51,51,51,51,51
East Village,100,100,100,100,100,100


In [8]:
print('There are {} unique food/restaurant types.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 124 unique food/restaurant types.


In [9]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Peking Duck Restaurant,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pizza Place,Poke Place,Portuguese Restaurant,Poutine Place,Puerto Rican Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Peking Duck Restaurant,Persian Restaurant,Peruvian Restaurant,Peruvian Roast Chicken Joint,Pet Café,Pizza Place,Poke Place,Portuguese Restaurant,Poutine Place,Puerto Rican Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Battery Park City,0.00,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.00,0.080000,0.000000,0.000000,0.00,0.040000,0.00,0.000000,0.080000,0.040000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.080000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.040000,0.040000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.160000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.120000,0.00,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
1,Carnegie Hill,0.00,0.000000,0.014706,0.000000,0.014706,0.000000,0.000000,0.00,0.000000,0.014706,0.073529,0.00,0.000000,0.00,0.000000,0.014706,0.000000,0.000000,0.102941,0.000000,0.00,0.000000,0.00,0.029412,0.000000,0.000000,0.00,0.014706,0.000000,0.044118,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.044118,0.000000,0.000000,0.000000,0.00,0.014706,0.000000,0.00,0.014706,0.00,0.029412,0.00,0.000000,0.000000,0.088235,0.000000,0.029412,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,0.00,0.00000,0.014706,0.029412,0.014706,0.000000,0.000000,0.00,0.014706,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.102941,0.000000,0.00,0.00,0.0000

In [11]:
num_top_venues = 10

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)




----Battery Park City----
                venue  freq
0         Pizza Place  0.16
1      Sandwich Place  0.12
2  Chinese Restaurant  0.08
3          Food Court  0.08
4        Burger Joint  0.08
5           BBQ Joint  0.08
6  Italian Restaurant  0.04
7          Steakhouse  0.04
8       Burrito Place  0.04
9          Donut Shop  0.04


----Carnegie Hill----
                  venue  freq
0                  Café  0.10
1           Pizza Place  0.10
2    Italian Restaurant  0.09
3      Sushi Restaurant  0.09
4                Bakery  0.07
5     French Restaurant  0.04
6                 Diner  0.04
7  Fast Food Restaurant  0.03
8     Indian Restaurant  0.03
9    Chinese Restaurant  0.03


----Central Harlem----
                             venue  freq
0                    Deli / Bodega  0.11
1              Fried Chicken Joint  0.09
2               African Restaurant  0.07
3               Seafood Restaurant  0.07
4  Southern / Soul Food Restaurant  0.07
5                      Pizza Place  0.07


                      venue  freq
0               Pizza Place  0.11
1        Italian Restaurant  0.08
2       American Restaurant  0.06
3                Bagel Shop  0.06
4          Asian Restaurant  0.05
5  Mediterranean Restaurant  0.05
6              Burger Joint  0.05
7         Indian Restaurant  0.05
8        Chinese Restaurant  0.03
9       Lebanese Restaurant  0.03


----Tribeca----
                 venue  freq
0  American Restaurant  0.14
1   Italian Restaurant  0.12
2                 Café  0.08
3        Deli / Bodega  0.06
4               Bakery  0.05
5         Burger Joint  0.05
6     Greek Restaurant  0.05
7       Sandwich Place  0.05
8           Poke Place  0.03
9          Pizza Place  0.03


----Tudor City----
                venue  freq
0       Deli / Bodega  0.12
1                Café  0.10
2  Mexican Restaurant  0.06
3         Pizza Place  0.06
4    Sushi Restaurant  0.05
5               Diner  0.05
6    Greek Restaurant  0.05
7     Thai Restaurant  0.04
8          Resta

In [12]:
#Now we have the TOP10 restaurants/food places for each neighbourhood

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Pizza Place,Sandwich Place,Food Court,Chinese Restaurant,BBQ Joint,Burger Joint,Bistro,Donut Shop,Italian Restaurant,Seafood Restaurant
1,Carnegie Hill,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,Bakery,Diner,French Restaurant,Restaurant,Taco Place,Vietnamese Restaurant
2,Central Harlem,Deli / Bodega,Fried Chicken Joint,African Restaurant,Pizza Place,Southern / Soul Food Restaurant,Caribbean Restaurant,Seafood Restaurant,Sandwich Place,American Restaurant,French Restaurant
3,Chelsea,Café,Bakery,French Restaurant,Pizza Place,American Restaurant,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Sandwich Place,Steakhouse
4,Chinatown,Chinese Restaurant,Bakery,Hotpot Restaurant,Dumpling Restaurant,Noodle House,Dim Sum Restaurant,Vegetarian / Vegan Restaurant,American Restaurant,Pizza Place,Mexican Restaurant
5,Civic Center,Italian Restaurant,American Restaurant,Sandwich Place,French Restaurant,Indian Restaurant,Bakery,Café,Deli / Bodega,Diner,Sushi Restaurant
6,Clinton,Italian Restaurant,Deli / Bodega,American Restaurant,Restaurant,Chinese Restaurant,Café,Sandwich Place,Thai Restaurant,Pizza Place,Food Court
7,East Harlem,Mexican Restaurant,Pizza Place,Bakery,Deli / Bodega,Café,Latin American Restaurant,Thai Restaurant,Donut Shop,Sandwich Place,Burger Joint
8,East Village,Pizza Place,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Korean Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Seafood Restaurant,American Restaurant
9,Financial District,Sandwich Place,Italian Restaurant,Salad Place,Food Truck,Pizza Place,Mexican Restaurant,American Restaurant,Café,Restaurant,Steakhouse


# Now we'll dive into the analysis for the italian restaurants
First we'll look into the neighbourhoods where Italian restaurants are the Number 1 restaurant, then we'll look into the areas where they don't appear in the TOP10. And finally, after the initial short list, we'll eliminate the neighbourhoods where "pizza place" is in the TOP10 to achieve a "clean" list of neighbourhoods with a supposed "untapped" potential for Italian restaurants

In [13]:
#selecting neighbourhoods where pizza or italian are number 1, as initial investigation of the market

italian1 = neighborhoods_venues_sorted.loc[(neighborhoods_venues_sorted['1st Most Common Venue'] == "Italian Restaurant")]

italian1


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Civic Center,Italian Restaurant,American Restaurant,Sandwich Place,French Restaurant,Indian Restaurant,Bakery,Café,Deli / Bodega,Diner,Sushi Restaurant
6,Clinton,Italian Restaurant,Deli / Bodega,American Restaurant,Restaurant,Chinese Restaurant,Café,Sandwich Place,Thai Restaurant,Pizza Place,Food Court
10,Flatiron,Italian Restaurant,Café,American Restaurant,Mexican Restaurant,Mediterranean Restaurant,Japanese Restaurant,New American Restaurant,Pizza Place,Salad Place,Sushi Restaurant
12,Greenwich Village,Italian Restaurant,Sushi Restaurant,Café,Pizza Place,Sandwich Place,Indian Restaurant,American Restaurant,French Restaurant,Japanese Restaurant,Chinese Restaurant
16,Lenox Hill,Italian Restaurant,Sushi Restaurant,Pizza Place,Burger Joint,Deli / Bodega,Café,Chinese Restaurant,American Restaurant,Mexican Restaurant,Bakery
18,Little Italy,Italian Restaurant,Chinese Restaurant,Bakery,Café,Mediterranean Restaurant,Pizza Place,Seafood Restaurant,Vietnamese Restaurant,French Restaurant,Sandwich Place
27,Noho,Italian Restaurant,Japanese Restaurant,Pizza Place,Mexican Restaurant,Sushi Restaurant,Café,French Restaurant,Bakery,Sandwich Place,Asian Restaurant
29,Soho,Italian Restaurant,Café,French Restaurant,Bakery,Sandwich Place,Mediterranean Restaurant,American Restaurant,Seafood Restaurant,Asian Restaurant,Pizza Place
34,Turtle Bay,Italian Restaurant,Steakhouse,Café,Deli / Bodega,Japanese Restaurant,Pizza Place,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Sandwich Place
35,Upper East Side,Italian Restaurant,American Restaurant,French Restaurant,Pizza Place,Diner,Bakery,Sushi Restaurant,Burger Joint,Mexican Restaurant,Sandwich Place


In [14]:
#let's now plot these neighbourghoods into the map
#first, we add the coordinates, then we plot the resulting dataframe


italian1_for_map = pd.merge(italian1, 
                      manhattan_data, 
                      on ='Neighborhood', 
                      how ='inner')
italian1_for_map




address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


# create map of Manhattan using latitude and longitude values
map_italian = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(italian1_for_map['Latitude'], italian1_for_map['Longitude'], italian1_for_map['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_italian) 
    
 
    
map_italian

In [15]:
#Now let's create a list of the neighbourds where Italian Restaurant does NOT show in the TOP10, as this will be our short list of neighbourhoods to install a new italian restaurant
italian2 = neighborhoods_venues_sorted.loc[(neighborhoods_venues_sorted['1st Most Common Venue'] != "Italian Restaurant") & (neighborhoods_venues_sorted['2nd Most Common Venue'] != "Italian Restaurant") & (neighborhoods_venues_sorted['3rd Most Common Venue'] != "Italian Restaurant") & (neighborhoods_venues_sorted['4th Most Common Venue'] != "Italian Restaurant") & (neighborhoods_venues_sorted['5th Most Common Venue'] != "Italian Restaurant")& (neighborhoods_venues_sorted['6th Most Common Venue'] != "Italian Restaurant")& (neighborhoods_venues_sorted['7th Most Common Venue'] != "Italian Restaurant")& (neighborhoods_venues_sorted['8th Most Common Venue'] != "Italian Restaurant")& (neighborhoods_venues_sorted['9th Most Common Venue'] != "Italian Restaurant")& (neighborhoods_venues_sorted['10th Most Common Venue'] != "Italian Restaurant")]

italian2


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Harlem,Deli / Bodega,Fried Chicken Joint,African Restaurant,Pizza Place,Southern / Soul Food Restaurant,Caribbean Restaurant,Seafood Restaurant,Sandwich Place,American Restaurant,French Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Hotpot Restaurant,Dumpling Restaurant,Noodle House,Dim Sum Restaurant,Vegetarian / Vegan Restaurant,American Restaurant,Pizza Place,Mexican Restaurant
7,East Harlem,Mexican Restaurant,Pizza Place,Bakery,Deli / Bodega,Café,Latin American Restaurant,Thai Restaurant,Donut Shop,Sandwich Place,Burger Joint
13,Hamilton Heights,Pizza Place,Deli / Bodega,Mexican Restaurant,Café,Latin American Restaurant,Sandwich Place,Chinese Restaurant,Donut Shop,Bakery,Caribbean Restaurant
15,Inwood,Restaurant,Pizza Place,Mexican Restaurant,Café,Bakery,Spanish Restaurant,Donut Shop,Diner,Latin American Restaurant,Deli / Bodega
19,Lower East Side,Deli / Bodega,Chinese Restaurant,Pizza Place,Bakery,Café,Diner,Fried Chicken Joint,Sandwich Place,Mexican Restaurant,Ramen Restaurant
22,Marble Hill,Deli / Bodega,American Restaurant,Sandwich Place,Bakery,Diner,Pizza Place,Seafood Restaurant,Snack Place,Food,Steakhouse
23,Midtown,American Restaurant,Japanese Restaurant,Sandwich Place,Steakhouse,Burger Joint,Café,French Restaurant,Sushi Restaurant,Bakery,Cuban Restaurant
24,Midtown South,Korean Restaurant,Bakery,Japanese Restaurant,Salad Place,American Restaurant,Café,New American Restaurant,Burger Joint,Ramen Restaurant,Vegetarian / Vegan Restaurant
25,Morningside Heights,Café,Deli / Bodega,Food Truck,Burger Joint,Pizza Place,American Restaurant,Sandwich Place,Chinese Restaurant,Mexican Restaurant,Indian Restaurant


In [22]:



italian2_for_map = pd.merge(italian2, 
                      manhattan_data, 
                      on ='Neighborhood', 
                      how ='inner')


address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


# create map of Manhattan using latitude and longitude values
map_italian = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
    
for lat, lng, label in zip(italian2_for_map['Latitude'], italian2_for_map['Longitude'], italian2_for_map['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_italian) 
    
 
    
map_italian

In [23]:
#let's now eliminate the neighbourhoods where "pizza place" is also in the TOP10 and display the remaining ones in the map

italian_pizza = italian2.loc[(italian2['1st Most Common Venue'] != "Pizza Place") & (italian2['2nd Most Common Venue'] != "Pizza Place") & (italian2['3rd Most Common Venue'] != "Pizza Place") & (italian2['4th Most Common Venue'] != "Pizza Place") & (italian2['5th Most Common Venue'] != "Pizza Place")& (italian2['6th Most Common Venue'] != "Pizza Place")& (italian2['7th Most Common Venue'] != "Pizza Place")& (italian2['8th Most Common Venue'] != "Pizza Place")& (italian2['9th Most Common Venue'] != "Pizza Place")& (italian2['10th Most Common Venue'] != "Pizza Place")]
italian_pizza


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Midtown,American Restaurant,Japanese Restaurant,Sandwich Place,Steakhouse,Burger Joint,Café,French Restaurant,Sushi Restaurant,Bakery,Cuban Restaurant
24,Midtown South,Korean Restaurant,Bakery,Japanese Restaurant,Salad Place,American Restaurant,Café,New American Restaurant,Burger Joint,Ramen Restaurant,Vegetarian / Vegan Restaurant
30,Stuyvesant Town,Deli / Bodega,Sandwich Place,German Restaurant,Bistro,Wings Joint,Filipino Restaurant,Food Truck,Food Court,Food,Falafel Restaurant


In [24]:
italian_pizza_for_map = pd.merge(italian_pizza, 
                      manhattan_data, 
                      on ='Neighborhood', 
                      how ='inner')


address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


# create map of Manhattan using latitude and longitude values
map_italian = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
    
for lat, lng, label in zip(italian_pizza_for_map['Latitude'], italian_pizza_for_map['Longitude'], italian_pizza_for_map['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_italian) 
    
 
    
map_italian

# And there it is! Knowing that the neighbourhood of Stuyvesant Town (bottom right) only got 5 restaurants in Foursquare (as per manhattan_venues.groupby('Neighborhood').count() results),  the neighbourhoods of Midtown and Midtown South look the most attractive to open an italian restaurant 